In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
#from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:

#input_df = pd.read_json('data.json', )
input_df_aes = pd.read_csv("doc_vector_for_aes.csv",)
target_df_aes = pd.read_csv("target_aes.csv")
#print(input_df.head())
input_df_aes = input_df_aes.iloc[:,1:]
target_df_aes = target_df_aes.iloc[:,1:]
print(input_df_aes.head())
print(input_df_aes.shape)
print(target_df_aes)
X = input_df_aes
Y = target_df_aes.to_numpy().T
print(type(Y))
#print(Y)

FileNotFoundError: [Errno 2] No such file or directory: 'doc_vector_for_aes.csv'

In [31]:
c_range = np.logspace(-2,10,10)
gamma_range = np.logspace(-10,5,10)
param_grid = dict( kernel= ['rbf'], C=c_range, gamma= gamma_range)
classifiers_aes = []
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.34, random_state=None)
train_index, test_index = [[]], [[]]
for train_index, test_index in sss.split(X, Y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    
    print(train_index, test_index)
    X_train, X_test = X.iloc[train_index, : ], X.iloc[test_index,:]
    y_train, y_test = Y[train_index], Y[test_index]
    
    #scaler = StandardScaler()
    #X_train = scaler.fit_transform(X_train)
    #X_test = scaler.fit_transform(X_test)
    
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.24, random_state=None)
    rbf_kernel_svm_clf = GridSearchCV(SVC(probability=True), param_grid=param_grid, cv = sss)
    rbf_kernel_svm_clf.fit(X_train, np.ravel(y_train,order='C'))
    classifiers_aes.append(rbf_kernel_svm_clf)
    print("The best parameters are %s with a score of %0.2f" % (rbf_kernel_svm_clf.best_params_, rbf_kernel_svm_clf.best_score_))
    
    y_hat_test = rbf_kernel_svm_clf.predict(X_test)
    y_hat_train = rbf_kernel_svm_clf.predict(X_train)
    #y_hat_test_proba = rbf_kernel_svm_clf.predict_proba(X_test)
    #y_hat_train_proba = rbf_kernel_svm_clf.predict_proba(X_train)
    
    
    #print(y_test.T)
    print(y_hat_test)
    print(y_hat_train)
    #print(y_train)
    test_acc = accuracy_score(y_test, y_hat_test)
    train_acc = accuracy_score(y_train, y_hat_train)
    print("test accuracy is {}, train accuracy is {}".format(test_acc, train_acc))
    
    
    

[16 20  9 74 42 96 24 26 98 53 78 85 87  1 10 13 37 33 97 51 45 14 18 50
 36 19 39 70 54 25  8 65 41 71 67 91 55 75 60 90 29 66 76 31 23 99 84  4
 72 58 82 68 49 89 28 73 43 86 95 92  5 17 44 83 93 15] [ 6  7 63 21 56 69 34 38 61 81 47 88 57 52  0 64 32 59 62 22  3 27 48 46
 79 94 11 77 40  2 12 80 35 30]
The best parameters are {'C': 4.6415888336127775, 'gamma': 0.021544346900318867, 'kernel': 'rbf'} with a score of 1.00
[1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 1]
[1 1 1 0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1]
test accuracy is 1.0, train accuracy is 1.0
[93 61 89  0  2 73 30 14 41  8 84 57 21 23 43 39 45 87 54 68 42 48 71 16
 49 95 28 98 46 79  4 76  9 37 10 64 86 56 34 94 96 17 90 62 51 55 59 11
 50 24  3 80 81  6 38 66 91 70 31 33 88 78 20 19 27 63] [13 58 44 29 83 97 82 25 47 32 75 22  7 85 52  1  5 72 69 74 40 67 36 92
 18 77 60 99 65 12 53 15 35 26]
The best parameter

In [100]:

#print(classifiers_aes)
train_output_prob = classifiers_aes[0].predict_proba(X)
total_sum = X.sum(axis=1)
print(total_sum)
print(classifiers_aes[0].predict(X))
print(train_output_prob)

0     256
1     256
2     256
3     256
4     256
     ... 
95     35
96     31
97     39
98     32
99     31
Length: 100, dtype: int64
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[0.03435663 0.96564337]
 [0.00777172 0.99222828]
 [0.0343744  0.9656256 ]
 [0.03443365 0.96556635]
 [0.00776435 0.99223565]
 [0.00776234 0.99223766]
 [0.03435229 0.96564771]
 [0.03434516 0.96565484]
 [0.00776487 0.99223513]
 [0.00776793 0.99223207]
 [0.00776255 0.99223745]
 [0.03428333 0.96571667]
 [0.03427812 0.96572188]
 [0.00776432 0.99223568]
 [0.00776947 0.99223053]
 [0.00776793 0.99223207]
 [0.00776585 0.99223415]
 [0.00774947 0.99225053]
 [0.00776238 0.99223762]
 [0.00776775 0.99223225]
 [0.00776585 0.99223415]
 [0.03443042 0.96556958]
 [0.03438995 0.96561005]
 [0.0077649  0.9922351 ]
 [0.00777737 0.99222263]
 [0.00776254 0.99223746]
 [0.007755

In [101]:
df_aes_output = pd.DataFrame()
#print(total_sum)
df_aes_output['sum_of_freq'] = total_sum
in_class_prob = []
out_of_class_prob = []
for item in train_output_prob :
    in_class_prob.append(item[1])
    out_of_class_prob.append(item[0])
df_aes_output['in_class_prob'] = in_class_prob
df_aes_output['out_of_class_prob'] = out_of_class_prob

#print(df_aes_output)
df_aes_output.to_csv('aes_train_data_prob.csv')

In [98]:
test_input_df_aes = pd.read_csv("test_doc_vector_for_aes.csv",)
test_target_df_aes = pd.read_csv("target_aes.csv")
test_input_df_aes = test_input_df_aes.iloc[:,1:]
test_target_df_aes = test_target_df_aes.iloc[:,1:]

print(test_input_df_aes.shape)
total_sum = test_input_df_aes.sum(axis=1)
print(total_sum)
test_target = test_target_df_aes.to_numpy().T
y_hat_test_target = classifiers_aes[0].predict(test_input_df_aes)
print("probability is {}".format(classifiers_aes[0].predict_proba(test_input_df_aes)))
test_output_prob = classifiers_aes[0].predict_proba(test_input_df_aes)
acc = accuracy_score(test_target,y_hat_test_target)
print(y_hat_test_target)
print("test accuracy is {}".format(acc))

(100, 8167)
0     33
1     18
2     27
3     32
4     26
      ..
95    32
96    25
97    31
98    29
99    30
Length: 100, dtype: int64
probability is [[0.9745285  0.0254715 ]
 [0.99884802 0.00115198]
 [0.99340733 0.00659267]
 [0.98451272 0.01548728]
 [0.99451057 0.00548943]
 [0.98223208 0.01776792]
 [0.95384961 0.04615039]
 [0.99679411 0.00320589]
 [0.99451478 0.00548522]
 [0.96672055 0.03327945]
 [0.9881661  0.0118339 ]
 [0.98850531 0.01149469]
 [0.9942446  0.0057554 ]
 [0.93011364 0.06988636]
 [0.99530378 0.00469622]
 [0.97835564 0.02164436]
 [0.99185916 0.00814084]
 [0.99028342 0.00971658]
 [0.97897267 0.02102733]
 [0.98689072 0.01310928]
 [0.99181518 0.00818482]
 [0.91804183 0.08195817]
 [0.98880161 0.01119839]
 [0.99171703 0.00828297]
 [0.99356791 0.00643209]
 [0.99622992 0.00377008]
 [0.95471617 0.04528383]
 [0.98846027 0.01153973]
 [0.97381449 0.02618551]
 [0.97866294 0.02133706]
 [0.93891012 0.06108988]
 [0.99432328 0.00567672]
 [0.94214628 0.05785372]
 [0.9889573  0.0110427 

In [99]:
df_test_aes_output = pd.DataFrame()
#print(total_sum)
df_test_aes_output['sum_of_freq'] = total_sum
in_class_prob = []
out_of_class_prob = []
print(test_output_prob)
for item in test_output_prob :
    in_class_prob.append(item[1])
    out_of_class_prob.append(item[0])
df_test_aes_output['in_class_prob'] = in_class_prob
df_test_aes_output['out_of_class_prob'] = out_of_class_prob

print(df_test_aes_output)
df_test_aes_output.to_csv('aes_test_data_prob.csv')

[[0.9745285  0.0254715 ]
 [0.99884802 0.00115198]
 [0.99340733 0.00659267]
 [0.98451272 0.01548728]
 [0.99451057 0.00548943]
 [0.98223208 0.01776792]
 [0.95384961 0.04615039]
 [0.99679411 0.00320589]
 [0.99451478 0.00548522]
 [0.96672055 0.03327945]
 [0.9881661  0.0118339 ]
 [0.98850531 0.01149469]
 [0.9942446  0.0057554 ]
 [0.93011364 0.06988636]
 [0.99530378 0.00469622]
 [0.97835564 0.02164436]
 [0.99185916 0.00814084]
 [0.99028342 0.00971658]
 [0.97897267 0.02102733]
 [0.98689072 0.01310928]
 [0.99181518 0.00818482]
 [0.91804183 0.08195817]
 [0.98880161 0.01119839]
 [0.99171703 0.00828297]
 [0.99356791 0.00643209]
 [0.99622992 0.00377008]
 [0.95471617 0.04528383]
 [0.98846027 0.01153973]
 [0.97381449 0.02618551]
 [0.97866294 0.02133706]
 [0.93891012 0.06108988]
 [0.99432328 0.00567672]
 [0.94214628 0.05785372]
 [0.9889573  0.0110427 ]
 [0.99219298 0.00780702]
 [0.99685354 0.00314646]
 [0.99319524 0.00680476]
 [0.99598688 0.00401312]
 [0.97177626 0.02822374]
 [0.99070895 0.00929105]


In [112]:

input_df_3des = pd.read_csv("doc_vector_for_3des.csv",)
target_df_3des = pd.read_csv("target_3des.csv")
#print(input_df.head())
input_df_3des = input_df_3des.iloc[:,1:]
target_df_3des = target_df_3des.iloc[:,1:]
print(input_df_3des.head())
print(input_df_3des.shape)
print(target_df_3des)
X = input_df_3des
Y = target_df_3des.to_numpy().T
print(type(Y))
#print(Y)

   aaad  aaan  aadb  aaee  aaei  aaeo  aaep  aafg  aafh  aafi  ...  pplj  \
0     0     0     0     0     0     0     0     0     0     0  ...     0   
1     0     0     0     0     0     0     0     0     0     0  ...     0   
2     0     1     0     0     0     0     0     0     0     0  ...     0   
3     0     0     0     0     0     0     0     0     0     0  ...     0   
4     0     0     0     0     0     0     0     0     0     0  ...     0   

   ppmb  ppmc  ppna  ppnb  ppnh  ppob  ppod  pppa  pppd  
0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0  

[5 rows x 7821 columns]
(100, 7821)
   0  1  2  3  4  5  6  7  8  9  ...  90  91  92  93  94  95  96  97  98  99
0  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0   0

[1 

In [65]:
c_range = np.logspace(-2,10,10)
gamma_range = np.logspace(-10,5,10)
param_grid = dict( kernel= ['rbf'], C=c_range, gamma= gamma_range)
classifiers_3des = []
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.34, random_state=None)

for train_index, test_index in sss.split(X, Y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = Y[train_index], Y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.24, random_state=None)
    rbf_kernel_svm_clf = GridSearchCV(SVC(probability=True), param_grid=param_grid, cv = sss)
    rbf_kernel_svm_clf.fit(X_train, np.ravel(y_train,order='C'))
    classifiers_3des.append(rbf_kernel_svm_clf)
    print("The best parameters are %s with a score of %0.2f" % (rbf_kernel_svm_clf.best_params_, rbf_kernel_svm_clf.best_score_))
    
    y_hat_test = rbf_kernel_svm_clf.predict(X_test)
    y_hat_train = rbf_kernel_svm_clf.predict(X_train)
    
    
    #print(y_test.T)
    print(y_hat_test)
    print(y_hat_train)
    #print(y_train)
    test_acc = accuracy_score(y_test, y_hat_test)
    train_acc = accuracy_score(y_train, y_hat_train)
    print("test accuracy is {}, train accuracy is {}".format(test_acc, train_acc))
    
    
    

The best parameters are {'C': 4.6415888336127775, 'gamma': 0.021544346900318867, 'kernel': 'rbf'} with a score of 1.00
[1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0]
[0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1
 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0]
test accuracy is 1.0, train accuracy is 1.0
The best parameters are {'C': 4.6415888336127775, 'gamma': 0.021544346900318867, 'kernel': 'rbf'} with a score of 1.00
[1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0]
[0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 1 1 1
 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1]
test accuracy is 1.0, train accuracy is 1.0
The best parameters are {'C': 4.6415888336127775, 'gamma': 0.021544346900318867, 'kernel': 'rbf'} with a score of 1.00
[0 1 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0]
[1 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1


In [109]:
train_output_prob = classifiers_3des[0].predict_proba(X)
total_sum = X.sum(axis=1)
print(total_sum)
print(classifiers_3des[0].predict(X))
print(train_output_prob)

0     37
1     27
2     30
3     41
4     32
      ..
95    28
96    28
97    36
98    34
99    20
Length: 100, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[0.94928925 0.05071075]
 [0.98861087 0.01138913]
 [0.98125105 0.01874895]
 [0.92016814 0.07983186]
 [0.98857944 0.01142056]
 [0.98858397 0.01141603]
 [0.99046645 0.00953355]
 [0.96996527 0.03003473]
 [0.96231873 0.03768127]
 [0.98857023 0.01142977]
 [0.98857104 0.01142896]
 [0.99018528 0.00981472]
 [0.98859795 0.01140205]
 [0.99296617 0.00703383]
 [0.98857001 0.01142999]
 [0.99205494 0.00794506]
 [0.98857408 0.01142592]
 [0.98858269 0.01141731]
 [0.99063132 0.00936868]
 [0.9885964  0.0114036 ]
 [0.98856978 0.01143022]
 [0.98858003 0.01141997]
 [0.98857012 0.01142988]
 [0.98858141 0.01141859]
 [0.94327177 0.05672823]
 [0.99029161 0.00970839]
 [0.98856206 0.011437

In [110]:
df_3des_output = pd.DataFrame()
#print(total_sum)
df_3des_output['sum_of_freq'] = total_sum
in_class_prob = []
out_of_class_prob = []
for item in train_output_prob :
    in_class_prob.append(item[1])
    out_of_class_prob.append(item[0])
df_3des_output['in_class_prob'] = in_class_prob
df_3des_output['out_of_class_prob'] = out_of_class_prob

#print(df_aes_output)
df_3des_output.to_csv('3des_train_data_prob.csv')

In [116]:
test_input_df_3des = pd.read_csv("test_doc_vector_for_3des.csv",)
test_target_df_3des = pd.read_csv("target_3des.csv")
test_input_df_3des = test_input_df_3des.iloc[:,1:]
test_target_df_3des = test_target_df_3des.iloc[:,1:]

print(test_input_df_3des.shape)
total_sum = test_input_df_3des.sum(axis=1)
print(total_sum)
test_target = test_target_df_3des.to_numpy().T
y_hat_test_target = classifiers_3des[0].predict(test_input_df_3des)
print("probability is {}".format(classifiers_3des[0].predict_proba(test_input_df_3des)))
test_output_prob = classifiers_3des[0].predict_proba(test_input_df_3des)
acc = accuracy_score(test_target,y_hat_test_target)
print(y_hat_test_target)
print("test accuracy is {}".format(acc))

(100, 7821)
0     26
1     32
2     26
3     32
4     30
      ..
95    30
96    45
97    37
98    34
99    29
Length: 100, dtype: int64
probability is [[0.99018241 0.00981759]
 [0.9738752  0.0261248 ]
 [0.99038661 0.00961339]
 [0.97504964 0.02495036]
 [0.98105536 0.01894464]
 [0.95451919 0.04548081]
 [0.98872533 0.01127467]
 [0.94771601 0.05228399]
 [0.9618469  0.0381531 ]
 [0.99609367 0.00390633]
 [0.96014464 0.03985536]
 [0.96142888 0.03857112]
 [0.97449439 0.02550561]
 [0.98125704 0.01874296]
 [0.98107621 0.01892379]
 [0.98999908 0.01000092]
 [0.98078226 0.01921774]
 [0.97835469 0.02164531]
 [0.99738585 0.00261415]
 [0.94994484 0.05005516]
 [0.96565069 0.03434931]
 [0.96611586 0.03388414]
 [0.98628174 0.01371826]
 [0.99030355 0.00969645]
 [0.94940819 0.05059181]
 [0.98895112 0.01104888]
 [0.99215567 0.00784433]
 [0.97401068 0.02598932]
 [0.98482323 0.01517677]
 [0.9962808  0.0037192 ]
 [0.97863039 0.02136961]
 [0.95354997 0.04645003]
 [0.94452438 0.05547562]
 [0.96244518 0.03755482

In [117]:
df_test_3des_output = pd.DataFrame()
#print(total_sum)
df_test_3des_output['sum_of_freq'] = total_sum
in_class_prob = []
out_of_class_prob = []
print(test_output_prob)
for item in test_output_prob :
    in_class_prob.append(item[1])
    out_of_class_prob.append(item[0])
df_test_3des_output['in_class_prob'] = in_class_prob
df_test_3des_output['out_of_class_prob'] = out_of_class_prob

print(df_test_3des_output)
df_test_3des_output.to_csv('3des_test_data_prob.csv')

[[0.99018241 0.00981759]
 [0.9738752  0.0261248 ]
 [0.99038661 0.00961339]
 [0.97504964 0.02495036]
 [0.98105536 0.01894464]
 [0.95451919 0.04548081]
 [0.98872533 0.01127467]
 [0.94771601 0.05228399]
 [0.9618469  0.0381531 ]
 [0.99609367 0.00390633]
 [0.96014464 0.03985536]
 [0.96142888 0.03857112]
 [0.97449439 0.02550561]
 [0.98125704 0.01874296]
 [0.98107621 0.01892379]
 [0.98999908 0.01000092]
 [0.98078226 0.01921774]
 [0.97835469 0.02164531]
 [0.99738585 0.00261415]
 [0.94994484 0.05005516]
 [0.96565069 0.03434931]
 [0.96611586 0.03388414]
 [0.98628174 0.01371826]
 [0.99030355 0.00969645]
 [0.94940819 0.05059181]
 [0.98895112 0.01104888]
 [0.99215567 0.00784433]
 [0.97401068 0.02598932]
 [0.98482323 0.01517677]
 [0.9962808  0.0037192 ]
 [0.97863039 0.02136961]
 [0.95354997 0.04645003]
 [0.94452438 0.05547562]
 [0.96244518 0.03755482]
 [0.97764899 0.02235101]
 [0.91855846 0.08144154]
 [0.96472885 0.03527115]
 [0.94704751 0.05295249]
 [0.94917928 0.05082072]
 [0.96487984 0.03512016]


In [118]:
input_df_blowfish = pd.read_csv("doc_vector_for_blowfish.csv",)
target_df_blowfish = pd.read_csv("target_blowfish.csv")
#print(input_df.head())
input_df_blowfish = input_df_blowfish.iloc[:,1:]
target_df_blowfish = target_df_blowfish.iloc[:,1:]
print(input_df_blowfish.head())
print(input_df_blowfish.shape)
print(target_df_blowfish)
X = input_df_blowfish
Y = target_df_blowfish.to_numpy().T
print(type(Y))
#print(Y)

   aaak  aaal  aaao  aabm  aach  aada  aadk  aaef  aaen  aafh  ...  ppim  \
0     0     0     0     0     0     0     0     0     0     0  ...     0   
1     0     0     0     0     0     0     0     0     0     0  ...     0   
2     0     0     0     0     0     0     0     0     0     0  ...     0   
3     0     0     0     0     0     0     0     0     0     0  ...     0   
4     0     0     0     0     0     0     0     0     0     0  ...     0   

   ppja  ppjf  ppkg  ppkj  pplg  ppll  ppmh  ppni  ppog  
0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0  

[5 rows x 7883 columns]
(100, 7883)
   0  1  2  3  4  5  6  7  8  9  ...  90  91  92  93  94  95  96  97  98  99
0  0  0  0  0  0  0  0  0  0  0  ...   1   1   1   1   1   1   1   1   1   1

[1 

In [68]:
c_range = np.logspace(-2,10,10)
gamma_range = np.logspace(-10,5,10)
param_grid = dict( kernel= ['rbf'], C=c_range, gamma= gamma_range)
classifiers_blowfish = []
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.34, random_state=None)

for train_index, test_index in sss.split(X, Y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = Y[train_index], Y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.24, random_state=None)
    rbf_kernel_svm_clf = GridSearchCV(SVC(probability = True), param_grid=param_grid, cv = sss)
    rbf_kernel_svm_clf.fit(X_train, np.ravel(y_train,order='C'))
    classifiers_blowfish.append(rbf_kernel_svm_clf)
    print("The best parameters are %s with a score of %0.2f" % (rbf_kernel_svm_clf.best_params_, rbf_kernel_svm_clf.best_score_))
    
    y_hat_test = rbf_kernel_svm_clf.predict(X_test)
    y_hat_train = rbf_kernel_svm_clf.predict(X_train)
    
    
    #print(y_test.T)
    print(y_hat_test)
    print(y_hat_train)
    #print(y_train)
    test_acc = accuracy_score(y_test, y_hat_test)
    train_acc = accuracy_score(y_train, y_hat_train)
    print("test accuracy is {}, train accuracy is {}".format(test_acc, train_acc))

The best parameters are {'C': 0.21544346900318834, 'gamma': 0.021544346900318867, 'kernel': 'rbf'} with a score of 1.00
[0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 0]
[0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0
 1 1 0 0 1 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 1]
test accuracy is 1.0, train accuracy is 1.0
The best parameters are {'C': 4.6415888336127775, 'gamma': 0.021544346900318867, 'kernel': 'rbf'} with a score of 1.00
[0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0]
[0 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 0
 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 1 0 1]
test accuracy is 1.0, train accuracy is 1.0
The best parameters are {'C': 4.6415888336127775, 'gamma': 0.021544346900318867, 'kernel': 'rbf'} with a score of 1.00
[1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1]
[1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0

In [119]:
train_output_prob = classifiers_blowfish[0].predict_proba(X)
total_sum = X.sum(axis=1)
print(total_sum)
print(classifiers_blowfish[0].predict(X))
print(train_output_prob)

0      26
1      25
2      25
3      37
4      32
     ... 
95    256
96    256
97    256
98    256
99    256
Length: 100, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[9.91491445e-01 8.50855451e-03]
 [9.93239983e-01 6.76001651e-03]
 [9.92697169e-01 7.30283128e-03]
 [9.80617084e-01 1.93829164e-02]
 [9.89250744e-01 1.07492563e-02]
 [9.89241328e-01 1.07586721e-02]
 [9.89242897e-01 1.07571035e-02]
 [9.96157395e-01 3.84260501e-03]
 [9.85990859e-01 1.40091408e-02]
 [9.89250723e-01 1.07492775e-02]
 [9.89239928e-01 1.07600722e-02]
 [9.65861125e-01 3.41388753e-02]
 [9.89234684e-01 1.07653159e-02]
 [9.89249759e-01 1.07502412e-02]
 [9.87026162e-01 1.29738376e-02]
 [9.89253257e-01 1.07467426e-02]
 [9.87856090e-01 1.21439095e-02]
 [9.84066338e-01 1.59336621e-02]
 [9.90986874e-01 9.01312646e-03]
 [9.36252822e-01 6.37471782e-02]


In [120]:
df_blowfish_output = pd.DataFrame()
#print(total_sum)
df_blowfish_output['sum_of_freq'] = total_sum
in_class_prob = []
out_of_class_prob = []
for item in train_output_prob :
    in_class_prob.append(item[1])
    out_of_class_prob.append(item[0])
df_blowfish_output['in_class_prob'] = in_class_prob
df_blowfish_output['out_of_class_prob'] = out_of_class_prob

#print(df_aes_output)
df_blowfish_output.to_csv('blowfish_train_data_prob.csv')

In [122]:
test_input_df_blowfish = pd.read_csv("test_doc_vector_for_blowfish.csv",)
test_target_df_blowfish = pd.read_csv("target_blowfish.csv")
test_input_df_blowfish = test_input_df_blowfish.iloc[:,1:]
test_target_df_blowfish = test_target_df_blowfish.iloc[:,1:]

print(test_input_df_blowfish.shape)
total_sum = test_input_df_blowfish.sum(axis=1)
print(total_sum)
test_target = test_target_df_blowfish.to_numpy().T
y_hat_test_target = classifiers_blowfish[0].predict(test_input_df_blowfish)
print("probability is {}".format(classifiers_blowfish[0].predict_proba(test_input_df_blowfish)))
test_output_prob = classifiers_blowfish[0].predict_proba(test_input_df_blowfish)
acc = accuracy_score(test_target,y_hat_test_target)
print(y_hat_test_target)
print("test accuracy is {}".format(acc))

(100, 7883)
0     19
1     29
2     23
3     29
4     37
      ..
95    26
96    29
97    32
98    35
99    33
Length: 100, dtype: int64
probability is [[0.9981503  0.0018497 ]
 [0.98345733 0.01654267]
 [0.99544092 0.00455908]
 [0.98395834 0.01604166]
 [0.93699311 0.06300689]
 [0.9665254  0.0334746 ]
 [0.97605645 0.02394355]
 [0.91196405 0.08803595]
 [0.96561922 0.03438078]
 [0.9895406  0.0104594 ]
 [0.99255677 0.00744323]
 [0.97133871 0.02866129]
 [0.95128696 0.04871304]
 [0.95241716 0.04758284]
 [0.97092848 0.02907152]
 [0.97656193 0.02343807]
 [0.92281656 0.07718344]
 [0.98663196 0.01336804]
 [0.96533914 0.03466086]
 [0.98919819 0.01080181]
 [0.94517212 0.05482788]
 [0.98348249 0.01651751]
 [0.97631026 0.02368974]
 [0.98063452 0.01936548]
 [0.98671955 0.01328045]
 [0.95868163 0.04131837]
 [0.85272926 0.14727074]
 [0.97695604 0.02304396]
 [0.98362925 0.01637075]
 [0.99630153 0.00369847]
 [0.99540669 0.00459331]
 [0.98921576 0.01078424]
 [0.94378152 0.05621848]
 [0.98436637 0.01563363

In [123]:
df_test_blowfish_output = pd.DataFrame()
#print(total_sum)
df_test_blowfish_output['sum_of_freq'] = total_sum
in_class_prob = []
out_of_class_prob = []
print(test_output_prob)
for item in test_output_prob :
    in_class_prob.append(item[1])
    out_of_class_prob.append(item[0])
df_test_blowfish_output['in_class_prob'] = in_class_prob
df_test_blowfish_output['out_of_class_prob'] = out_of_class_prob

print(df_test_blowfish_output)
df_test_blowfish_output.to_csv('blowfish_test_data_prob.csv')

[[0.9981503  0.0018497 ]
 [0.98345733 0.01654267]
 [0.99544092 0.00455908]
 [0.98395834 0.01604166]
 [0.93699311 0.06300689]
 [0.9665254  0.0334746 ]
 [0.97605645 0.02394355]
 [0.91196405 0.08803595]
 [0.96561922 0.03438078]
 [0.9895406  0.0104594 ]
 [0.99255677 0.00744323]
 [0.97133871 0.02866129]
 [0.95128696 0.04871304]
 [0.95241716 0.04758284]
 [0.97092848 0.02907152]
 [0.97656193 0.02343807]
 [0.92281656 0.07718344]
 [0.98663196 0.01336804]
 [0.96533914 0.03466086]
 [0.98919819 0.01080181]
 [0.94517212 0.05482788]
 [0.98348249 0.01651751]
 [0.97631026 0.02368974]
 [0.98063452 0.01936548]
 [0.98671955 0.01328045]
 [0.95868163 0.04131837]
 [0.85272926 0.14727074]
 [0.97695604 0.02304396]
 [0.98362925 0.01637075]
 [0.99630153 0.00369847]
 [0.99540669 0.00459331]
 [0.98921576 0.01078424]
 [0.94378152 0.05621848]
 [0.98436637 0.01563363]
 [0.9804723  0.0195277 ]
 [0.98415519 0.01584481]
 [0.98354571 0.01645429]
 [0.97034531 0.02965469]
 [0.96724864 0.03275136]
 [0.97583964 0.02416036]


In [2]:

#input_df = pd.read_json('data.json', )
input_df_aes = pd.read_csv("doc_vector_for_aes.csv",)
target_df_aes = pd.read_csv("target_aes.csv")
#print(input_df.head())
input_df_aes = input_df_aes.iloc[:,1:]
target_df_aes = target_df_aes.iloc[:,1:]
print(input_df_aes.head())
print(input_df_aes.shape)
print(target_df_aes)
X = input_df_aes
Y = target_df_aes.to_numpy().T
print(type(Y))
#print(Y)

   aaac  aaae  aaan  aaao  aabl  aacb  aacc  aacd  aado  aaeh  ...  pplg  \
0     0     0     0     0     0     0     0     0     0     0  ...     0   
1     0     0     0     0     0     0     0     0     0     0  ...     0   
2     0     0     1     0     0     0     0     0     0     0  ...     0   
3     0     0     0     0     0     0     0     0     0     0  ...     0   
4     0     0     0     0     0     0     0     0     0     0  ...     0   

   ppll  ppln  pplp  ppmd  ppob  ppol  ppop  ppph  pppi  
0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0  

[5 rows x 8167 columns]
(100, 8167)
   0  1  2  3  4  5  6  7  8  9  ...  90  91  92  93  94  95  96  97  98  99
0  1  1  1  1  1  1  1  1  1  1  ...   0   0   0   0   0   0   0   0   0   0

[1 

In [ ]:
c_range = np.logspace(-2,10,10)
gamma_range = np.logspace(-10,5,10)
param_grid = dict( kernel= ['rbf'], C=c_range, gamma= gamma_range)
classifiers_aes = []
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.34, random_state=None)
train_index, test_index = [[]], [[]]
for train_index, test_index in sss.split(X, Y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    
    print(train_index, test_index)
    X_train, X_test = X.iloc[train_index, : ], X.iloc[test_index,:]
    y_train, y_test = Y[train_index], Y[test_index]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.24, random_state=None)
    rbf_kernel_svm_clf = GridSearchCV(SVC(probability=True), param_grid=param_grid, cv = sss)
    rbf_kernel_svm_clf.fit(X_train, np.ravel(y_train,order='C'))
    classifiers_aes.append(rbf_kernel_svm_clf)
    print("The best parameters are %s with a score of %0.2f" % (rbf_kernel_svm_clf.best_params_, rbf_kernel_svm_clf.best_score_))
    
    y_hat_test = rbf_kernel_svm_clf.predict(X_test)
    y_hat_train = rbf_kernel_svm_clf.predict(X_train)
    #y_hat_test_proba = rbf_kernel_svm_clf.predict_proba(X_test)
    #y_hat_train_proba = rbf_kernel_svm_clf.predict_proba(X_train)
    
    
    #print(y_test.T)
    print(y_hat_test)
    print(y_hat_train)
    #print(y_train)
    test_acc = accuracy_score(y_test, y_hat_test)
    train_acc = accuracy_score(y_train, y_hat_train)
    print("test accuracy is {}, train accuracy is {}".format(test_acc, train_acc))
    
    
    

[17 22  2 19 57 44 94  8 48 73 30 31 36 33 18 45 82 38 43  0 39 59 92 26
  5 63 34 91 60 96 93 75 71 88 98 28 35 74 97 47 78 81 70 87 46  7 99 15
 69 40  3 61 14 52 10  4 77 85 84 68 50  6 37 53 29 20] [49 83 41 24 25 89 67 95 86 58  1 32 27 79 80 42 13 76 64 11 62 54 66 65
 72  9 55 12 16 21 90 23 56 51]


In [ ]:

print(classifiers_aes[0].predict(X))